In [ ]:
from tvb.simulator.simulator import Simulator
from tvb.datatypes.connectivity import Connectivity
from tvb.contrib.inversion.sbiInference import sbiModel

import tvb.simulator.models
import tvb.simulator.integrators
import tvb.simulator.coupling
import tvb.simulator.monitors

import matplotlib.pyplot as plt
import numpy as np
import torch
import math
import arviz as az
import pickle

%load_ext autoreload
%autoreload 2

In [ ]:
#run_ids = [
#    "2022-08-02_17-39-15-535669_instance",
#    "2022-08-02_17-39-15-536677_instance",
#    "2022-08-02_17-39-15-560830_instance",
#    "2022-08-02_17-39-15-566932_instance",
#    "2022-08-02_17-39-15-656787_instance",
#    "2022-08-02_17-39-15-662583_instance"
#]

#run_ids = [
#    "2022-08-02_18-11-00-914584_instance",
#    "2022-08-02_18-11-00-915996_instance",
#    "2022-08-02_18-11-00-916394_instance",
#    "2022-08-02_18-11-00-928621_instance",
#    "2022-08-02_18-11-00-960091_instance",
#    "2022-08-02_18-11-00-964377_instance"
#]

#run_ids = [
#    "2022-08-04_14-38-33-259417_instance",
#    "2022-08-04_14-38-33-266844_instance",
#    "2022-08-04_14-38-33-267835_instance",
#    "2022-08-04_14-38-33-268056_instance",
#    "2022-08-04_14-38-33-270680_instance",
#    "2022-08-04_14-38-33-280629_instance"
#]

#run_ids = [
#    "2022-08-05_11-7-09-822425_instance",
#    "2022-08-05_11-7-09-825593_instance",
#    "2022-08-05_11-7-09-829123_instance",
#    "2022-08-05_11-7-09-833147_instance",
#    "2022-08-05_11-7-09-846828_instance",
#    "2022-08-05_11-7-09-857880_instance"
#]

#run_ids = [
#    "2022-08-05_12-7-35-719303_instance",
#    "2022-08-05_12-7-35-723258_instance",
#    "2022-08-05_12-7-35-731066_instance",
#    "2022-08-05_12-7-35-751849_instance",
#    "2022-08-05_12-7-35-765617_instance",
#    "2022-08-05_12-7-35-778611_instance"
#]

#run_ids = [
#    "2022-08-05_14-24-57-808939_instance",
#    "2022-08-05_14-24-57-819499_instance",
#    "2022-08-05_14-24-57-828211_instance",
#    "2022-08-05_14-24-57-843612_instance",
#    "2022-08-05_14-24-57-850003_instance",
#    "2022-08-05_14-24-57-861756_instance"
#]

#run_ids = [
#    "2022-08-12_09-21-32-004479_instance",
#    "2022-08-12_09-21-32-058462_instance",
#    "2022-08-12_09-21-32-159366_instance",
#    "2022-08-12_09-21-32-164984_instance",
#    "2022-08-12_09-21-32-166316_instance",
#    "2022-08-12_09-21-32-180884_instance"
#]

#run_ids = [
#    "2022-08-25_15-24-36-281214_instance",
#    "2022-08-25_15-24-36-282809_instance",
#    "2022-08-25_15-24-36-297549_instance",
#    "2022-08-25_15-24-36-311166_instance",
#    "2022-08-25_15-24-36-316330_instance",
#    "2022-08-25_15-24-36-331989_instance"
#]

#run_ids = [
#    "2022-08-28_19-9-48-122098_instance",
#    "2022-08-28_19-9-48-122101_instance",
#    "2022-08-28_19-9-48-132533_instance",
#    "2022-08-28_19-9-48-158329_instance",
#    "2022-08-28_19-9-48-162346_instance",
#    "2022-08-28_19-9-48-178139_instance"
#]

run_ids = [
    "2022-08-28_22-46-19-852895_instance",
    "2022-08-28_22-46-19-920016_instance",
    "2022-08-28_22-46-19-920017_instance",
    "2022-08-28_22-46-19-920119_instance",
    "2022-08-28_22-46-20-000723_instance",
    "2022-08-28_22-46-20-003268_instance"
]

#run_ids = [
#    "2022-08-30_13-55-36-421593_instance",
#    "2022-08-30_13-55-36-425459_instance",
#    "2022-08-30_13-55-36-436073_instance",
#    "2022-08-30_13-55-36-453707_instance",
#    "2022-08-30_13-55-36-457131_instance",
#    "2022-08-30_13-55-36-465106_instance"
#]

#run_ids = [
#    "2022-08-30_14-45-19-687012_instance",
#    "2022-08-30_14-45-19-695750_instance",
#    "2022-08-30_14-45-19-712372_instance",
#    "2022-08-30_14-45-19-717827_instance",
#    "2022-08-30_14-45-19-733389_instance",
#    "2022-08-30_14-45-19-734220_instance"
#]

#run_ids = [
#    "2022-08-30_16-34-42-798455_instance",
#    "2022-08-30_16-34-42-798462_instance",
#    "2022-08-30_16-34-42-799973_instance",
#    "2022-08-30_16-34-42-801938_instance",
#    "2022-08-30_16-34-42-809439_instance",
#    "2022-08-30_16-34-42-813425_instance"
#]

#run_ids = [
#    "2022-08-31_17-41-41-107233_instance",
#    "2022-08-31_17-41-41-126333_instance",
#    "2022-08-31_17-41-41-138178_instance",
#    "2022-08-31_17-41-41-140905_instance",
#    "2022-08-31_17-41-41-153404_instance",
#    "2022-08-31_17-41-41-164193_instance"
#]

#run_ids = [
#    "2022-08-31_19-29-22-263454_instance",
#    "2022-08-31_19-29-22-266365_instance",
#    "2022-08-31_19-29-22-275650_instance",
#    "2022-08-31_19-29-22-275772_instance",
#    "2022-08-31_19-29-22-285128_instance",
#    "2022-08-31_19-29-22-301773_instance"
#]

In [ ]:
with open(f"sbi_data/inference_data/{run_ids[0]}.pkl", "rb") as f:
    instance_params = pickle.load(f)
    simulation_params = instance_params["simulation_params"]

In [ ]:
# Connectivity
if simulation_params["connectivity"] == "Own":
    connectivity = Connectivity()
    connectivity.weights = np.array([[0., 2/3], [2/3, 0.]])
    connectivity.region_labels = np.array(["R1", "R2"])
    connectivity.centres = np.array([[0.1, 0.1, 0.1], [0.2, 0.1, 0.1]])
    connectivity.tract_lengths = np.array([[0., 0.1], [0.1, 0.]])
    connectivity.configure()

# Model
oscillator_model = getattr(tvb.simulator.models, simulation_params["model"])(
    a=np.asarray([simulation_params["a_sim"]]),
    b=np.asarray([simulation_params["b_sim"]]),
    c=np.asarray([simulation_params["c_sim"]]),
    d=np.asarray([simulation_params["d_sim"]]),
    I=np.asarray([simulation_params["I_sim"]]),
)
oscillator_model.configure()

# Integrator
integrator = getattr(tvb.simulator.integrators, simulation_params["integrator"])(dt=simulation_params["dt"])
integrator.noise.nsig = np.array([simulation_params["nsig"]])
integrator.configure()

# Global coupling
coupling = getattr(tvb.simulator.coupling, simulation_params["coupling"])()

# Monitor
monitor = getattr(tvb.simulator.monitors, simulation_params["monitor"])()

In [ ]:
# Simulator
sim = Simulator(
    model=oscillator_model,
    connectivity=connectivity,
    coupling=coupling,
    integrator=integrator,
    monitors=(monitor,),
    simulation_length=simulation_params["simulation_length"]
)

sim.configure()

In [ ]:
X = instance_params["obs"]

In [ ]:
f1 = plt.figure(figsize=(14,8))
plt.plot(X[:, 0, 0, 0], label="Region 1")
plt.plot(X[:, 0, 1, 0], label="Region 2")
plt.ylabel("states", fontsize=16)
plt.xlabel("time (ms)", fontsize=16)
plt.legend(fontsize=16)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.show()

### SNPE inference

In [ ]:
snpe_model = sbiModel(
    simulator_instance=sim,
    method="SNPE", 
    obs=X
)

In [ ]:
#criteria = []
#for i in run_ids:
#    snpe_model.load(f"{i}.pkl")
#    criteria.append(snpe_model.information_criteria())

In [ ]:
snpe_model.load(f"{run_ids[5]}.pkl")

In [ ]:
snpe_model.plot_posterior_samples(
    init_params={"a_model": simulation_params["a_sim"], 
                 "b_model": simulation_params["b_sim"],
                 "c_model": simulation_params["c_sim"],
                 "d_model": simulation_params["d_sim"],
                 "I_model": simulation_params["I_sim"],
                 "a_coupling": 0.1,
                 "nsig_integrator.noise": 0.003,
                 "epsilon_global": 0.0}
)

In [ ]:
print("posterior std dev:", snpe_model.posterior_samples.std(dim=0).numpy())
print("prior std dev:", torch.diag(snpe_model.priors.scale_tril).numpy())
print("shrinkages:", snpe_model.posterior_shrinkage().numpy())

In [ ]:
f2 = plt.figure(figsize=(12,8))
plt.plot(snpe_model.posterior_shrinkage(), snpe_model.posterior_zscore(), 
         color="blue", linewidth=0, marker="*", markersize=12)

In [ ]:
posterior_sample = snpe_model.get_sample()[0]
posterior_sample

In [ ]:
posterior_obs = snpe_model.simulation_wrapper(params=posterior_sample)
posterior_obs = posterior_obs.numpy().reshape(X.shape, order="F")

In [ ]:
f3, axes3 = plt.subplots(nrows=2, ncols=1, figsize=(18,15))
axes3[0].plot(snpe_model.inference_data.observed_data.x_obs.values[:, 0, 0, 0], label="R1_observed", color="blue")
axes3[0].plot(posterior_obs[:, 0, 0, 0], label="R1_posterior", color="cyan")
axes3[0].legend(fontsize=16)
axes3[0].set_xlabel("time (ms)", fontsize=16)
axes3[0].tick_params(axis="both", labelsize=16)

axes3[1].plot(snpe_model.inference_data.observed_data.x_obs.values[:, 0, 1, 0], label="R2_observed", color="red")
axes3[1].plot(posterior_obs[:, 0, 1, 0], label="R2_posterior", color="orange")
axes3[1].legend(fontsize=16)
axes3[1].set_xlabel("time (ms)", fontsize=16)
axes3[1].tick_params(axis="both", labelsize=16)

plt.show()

In [ ]:
#snpe_model.save()

### SNLE inference

In [ ]:
snle_model = sbiModel(
    simulator_instance=sim,
    method="SNLE", 
    obs=obs, 
    prior_vars=priors,
    prior_dist="Normal",
)

In [ ]:
snle_model.run_inference(
    num_simulations=800,
    num_workers=4,
    num_samples=2000
)

In [ ]:
inference_data = snle_model.to_arviz_data(save=True)

In [ ]:
snle_model.plot_posterior_samples(
    init_params={"a": a_sim, "b": b_sim, "c": c_sim, "d": d_sim, "I": I_sim, "epsilon": 0.0},
    bins=50
)

In [ ]:
map_estimator = snle_model.get_map_estimator()
map_estimator

In [ ]:
posterior_sample = snle_model.get_sample()[0]
posterior_sample

In [ ]:
posterior_obs = snle_model.simulation_wrapper(params=posterior_sample)
posterior_obs = posterior_obs.numpy().reshape(shape, order="F")

In [ ]:
f4 = plt.figure(figsize=(13,8))
plt.plot(snle_model.inference_data.observed_data.x_obs.values[:, 0, 0, 0], label="V_simulated", color="blue")
plt.plot(snle_model.inference_data.observed_data.x_obs.values[:, 1, 0, 0], label="W_simulated", color="red")
plt.plot(posterior_obs[:, 0, 0, 0], label="V_posterior", color="cyan")
plt.plot(posterior_obs[:, 1, 0, 0], label="W_posterior", color="orange")
plt.legend()
plt.xlabel("time (ms)")
plt.ylabel("states")
plt.show()

In [ ]:
snle_model.information_criteria()

### SNRE inference

In [ ]:
snre_model = sbiModel(
    integrator_instance=integrator, 
    model_instance=oscillator_model, 
    method="SNRE", 
    obs=obs, 
    priors=priors,
    obs_shape=shape
)

In [ ]:
snre_model.run_inference(
    num_simulations=800,
    num_workers=1,
    num_samples=2000
)

In [ ]:
inference_data = snre_model.to_arviz_data(save=True)

In [ ]:
snre_model.plot_posterior(
    init_params={"a": a_sim, "b": b_sim, "c": c_sim, "d": d_sim, "I": I_sim, "epsilon": 0.0}
)

In [ ]:
map_estimator = snre_model.get_map_estimator()
map_estimator

In [ ]:
posterior_sample = snre_model.get_sample()[0]
posterior_sample

In [ ]:
posterior_obs = snre_model.simulation_wrapper(params=map_estimator)
posterior_obs = posterior_obs.numpy().reshape(shape, order="F")

In [ ]:
f4 = plt.figure(figsize=(13,8))
plt.plot(snre_model.inference_data.observed_data.x_obs.values[:, 0, 0, 0], label="V_simulated", color="blue")
plt.plot(snre_model.inference_data.observed_data.x_obs.values[:, 1, 0, 0], label="W_simulated", color="red")
plt.plot(posterior_obs[:, 0, 0, 0], label="V_posterior", color="cyan")
plt.plot(posterior_obs[:, 1, 0, 0], label="W_posterior", color="orange")
plt.legend()
plt.xlabel("time (ms)")
plt.ylabel("states")
plt.show()

In [ ]:
snre_model.information_criteria()